<a href="https://colab.research.google.com/github/uzairah-206/LangChain/blob/main/LANGCHAIN_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##1. Introduction to Retrieval-Augmented Generation (RAG)

RAG combines retrieval-based methods (fetching relevant documents) with generative models (like GPT) to produce more accurate and context-aware responses.

Key Concepts:
Retrieval: Fetching relevant documents from a knowledge base.

Generation: Using a language model to generate answers based on retrieved documents.

In [9]:
!pip install -U langchain-community
!pip install faiss-cpu

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.7/30.7 MB 30.7 MB/s eta 0:00:00


In [ ]:
from langchain.chains import RetrievalQA
from langchain.document_loaders import TextLoader
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFaceHub
from langchain.text_splitter import RecursiveCharacterTextSplitter

# Load a document
loader = TextLoader("example.txt")
documents = loader.load()

# Split documents into chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)
texts = text_splitter.split_documents(documents)

# Create embeddings and store in a vector database
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embeddings)

# Initialize a retrieval chain
llm = HuggingFaceHub(repo_id="google/flan-t5-small")
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

# Query the chain
query = "What is the main topic of the document?"
result = qa_chain.run(query)
print(result)

##2. Document Loading in LangChain


In [4]:
from langchain.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://www.google.com")
data = loader.load()
print(data)

[Document(metadata={'source': 'https://www.google.com', 'title': 'Google', 'description': "Search the world's information, including webpages, images, videos and more. Google has many special features to help you find exactly what you're looking for.", 'language': 'en'}, page_content='GoogleSearch Images Maps Play YouTube News Gmail Drive More »Web History | Settings | Sign in\xa0Advanced searchAdvertisingBusiness SolutionsAbout Google© 2025 - Privacy - Terms   ')]


##3. Document Splitting and Chunking


In [7]:
"""Documents are often too large for language models, so they need to be split into smaller chunks."""
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=50,
    length_function=len,
)
texts = text_splitter.split_documents(data)
print(texts)

[Document(metadata={'source': 'https://www.google.com', 'title': 'Google', 'description': "Search the world's information, including webpages, images, videos and more. Google has many special features to help you find exactly what you're looking for.", 'language': 'en'}, page_content='GoogleSearch Images Maps Play YouTube News Gmail Drive More »Web History | Settings | Sign in\xa0Advanced searchAdvertisingBusiness SolutionsAbout Google© 2025 - Privacy - Terms')]


##4. Using Vector Databases for Retrieval


In [11]:
"""Vector databases (like FAISS, Pinecone, or Weaviate) store embeddings for efficient retrieval."""
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embeddings)

query = "What is the main topic of the document?"
docs = vectorstore.similarity_search(query)
print(docs[0].page_content)

GoogleSearch Images Maps Play YouTube News Gmail Drive More »Web History | Settings | Sign in Advanced searchAdvertisingBusiness SolutionsAbout Google© 2025 - Privacy - Terms


##5. Embedding and Storing Documents


In [ ]:
"""Embeddings convert text into numerical vectors for storage and retrieval."""
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(texts, embeddings)

##6. Constructing a Retrieval Chain


In [13]:
#Combine retrieval and generation into a single chain.

from langchain.chains import RetrievalQA
from langchain.llms import HuggingFaceHub

llm = HuggingFaceHub(repo_id="google/flan-t5-small")
qa_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=vectorstore.as_retriever())

#query the chain
query = "What is the main topic of the document?"
result = qa_chain.run(query)
print(result)

<ipython-input-13-7549d564d8c0>:6: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEndpoint``.
  llm = HuggingFaceHub(repo_id="google/flan-t5-small")


ValidationError: 1 validation error for HuggingFaceHub
  Value error, Did not find huggingfacehub_api_token, please add an environment variable `HUGGINGFACEHUB_API_TOKEN` which contains it, or pass `huggingfacehub_api_token` as a named parameter. [type=value_error, input_value={'repo_id': 'google/flan-...acehub_api_token': None}, input_type=dict]
    For further information visit https://errors.pydantic.dev/2.10/v/value_error

##7.Building Advanced RAG Architectures

💡 Concept:

In advanced RAG architectures, the idea is to integrate various sophisticated retrieval and generation techniques to answer complex queries.

You might involve multiple retrieval steps (e.g., querying different databases) and combine information from various models to generate comprehensive answers.

In [12]:
#For advanced RAG, you can chain multiple retrieval steps and generate different types of responses. For example, querying documents, followed by a retrieval from a knowledge base or external source like a database or API.

from langchain.chains import RetrievalQA
from langchain.llms import HuggingFacePipeline
from langchain.vectorstores import FAISS
# Create a second retriever
retriever_2 = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 5})

# Combine multiple retrievers into one
combined_retriever = retriever_2  # Add more retrievers if needed

# Advanced RAG Chain
qa_chain_advanced = RetrievalQA.from_chain_type(llm, retriever=combined_retriever)
answer_advanced = qa_chain_advanced.run("What is LangChain?")
print(answer_advanced)

NameError: name 'llm' is not defined

##9.Understanding Vector Stores and Embeddings

💡 Concept:

Vector Stores are the systems where embeddings (numerical representations of text) are stored and indexed.

Embeddings convert text into high-dimensional vectors that can be used for similarity search and information

retrieval.

FAISS is a highly optimized library for fast nearest neighbor search in large datasets.

In [ ]:
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS

# Embed documents
embedding_model = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vectorstore = FAISS.from_documents(documents, embedding_model)

# Search for similar documents
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs = retriever.get_relevant_documents("What is the future of AI?")
print(retrieved_docs)


##10. Improving Retrieval with Sparse and Dense Methods

💡 Concept:

Dense retrieval uses embedding-based methods to search for documents. It’s fast and works well with unstructured text.

Sparse retrieval (e.g., TF-IDF, BM25) relies on traditional keyword-based matching and is more effective in highly structured or highly detailed documents.

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Sparse method (TF-IDF)
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform([doc.page_content for doc in docs])
similarities = cosine_similarity(tfidf_matrix, tfidf_matrix[0])  # Compare first doc to others

# Dense retrieval (FAISS)
retriever = vectorstore.as_retriever(search_type="similarity", search_kwargs={"k": 3})
retrieved_docs_dense = retriever.get_relevant_documents("Explain RAG")


##11.Optimizing Document Splitting with Semantic Chunking

💡 Concept:

Document splitting refers to dividing large documents into smaller chunks for easier processing.

Semantic Chunking considers the meaning and context of the document rather than just fixed-size chunks to improve relevance during retrieval.

In [ ]:
from langchain.text_splitter import SemanticTextSplitter

# Use a semantic text splitter instead of the standard character-based one
splitter = SemanticTextSplitter(chunk_size=500, chunk_overlap=50)
chunks = splitter.split_documents(documents)

"""Semantic Chunking helps ensure chunks are contextually meaningful, improving the quality of retrieval."""

##12.Introduction to Graph-Based RAG

💡 Concept:

Graph-Based RAG uses graphs to represent complex relationships between documents or entities. It enhances information retrieval by taking advantage of the
interconnectivity of data.

Neo4j is a popular graph database that stores entities and relationships.

In [15]:
!pip install neo4j

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 312.3/312.3 kB 5.5 MB/s eta 0:00:00


In [16]:
from neo4j import GraphDatabase

# Connect to Neo4j
uri = "bolt://localhost:7687"
driver = GraphDatabase.driver(uri, auth=("neo4j", "password"))

# Query data
def get_node_data(tx, topic):
    result = tx.run("MATCH (n:Topic {name: $topic}) RETURN n", topic=topic)
    return result.single()

with driver.session() as session:
    node_data = session.read_transaction(get_node_data, "RAG")
    print(node_data)


<ipython-input-16-ac2a50a475ab>:13: DeprecationWarning: read_transaction has been renamed to execute_read
  node_data = session.read_transaction(get_node_data, "RAG")
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 99] Cannot assign requested address))
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 99] Cannot assign requested address))
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connection refused)
Failed to establish connection to ResolvedIPv6Address(('::1', 7687, 0, 0)) (reason [Errno 99] Cannot assign requested address))
Failed to establish connection to ResolvedIPv4Address(('127.0.0.1', 7687)) (reason [Errno 111] Connectio

KeyboardInterrupt: 

##13.Generating Cypher Queries with LLMs

LLMs (Large Language Models) can generate Cypher queries for querying graph databases like Neo4j.

In [17]:
from transformers import pipeline

# Generate Cypher queries using LLM
generator = pipeline("text2text-generation", model="facebook/bart-large-cnn")
query = generator("Generate a Cypher query to find related topics about RAG")[0]['generated_text']
print(query)


config.json:   0%|          | 0.00/1.58k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.63G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/363 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

Device set to use cpu


Generate a Cypher query to find related topics about RAG. Generate an RAG-related Cypher question to find out more about the RAG language. Use these questions to help you understand RAG in your own language. For more information on RAG, visit RAG's website.


##14.Implementing Graph RAG Chains

💡 Concept:

Graph RAG combines document retrieval with graph-based data to enhance the retrieval process.

Neo4j's nodes and relationships can guide the agent to better search for relevant information across interconnected data.

##15.Enhancing Graph RAG with Validation and Filtering

After retrieving information from a graph or a document database, validation ensures that the information is relevant, and filtering removes irrelevant data before sending it to the generation model.

##16.Introduction to AI Agents and Tools

💡 Concept:

AI Agents can perform specific tasks (e.g., answering questions, browsing the web).

LangChain provides tools to integrate various external APIs, tools, and memory into agents to handle dynamic requests.
